In [5]:
# import libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2
from sklearn.preprocessing import KBinsDiscretizer
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# Load the data
df = pd.read_csv("Social_Vulnerability_Index_2018_-_United_States__tract_20250119.csv")

# Check the data overview
print("Column names:")
print(df.columns)

print("\nSummary of 'RPL_THEMES' before preprocessing:")
print(df["RPL_THEMES"].describe())

Column names:
Index(['ST', 'STATE', 'ST_ABBR', 'COUNTY', 'FIPS', 'LOCATION', 'AREA_SQMI',
       'E_TOTPOP', 'M_TOTPOP', 'E_HU',
       ...
       'F_NOVEH', 'F_GROUPQ', 'F_THEME4', 'F_TOTAL', 'E_UNINSUR', 'M_UNINSUR',
       'EP_UNINSUR', 'MP_UNINSUR', 'E_DAYPOP', 'Shape'],
      dtype='object', length=124)

Summary of 'RPL_THEMES' before preprocessing:
count    72837.000000
mean        -8.611694
std         94.996397
min       -999.000000
25%          0.243100
50%          0.495400
75%          0.747700
max          1.000000
Name: RPL_THEMES, dtype: float64


In [7]:
# Keep and copy data where 'RPL_THEMES' is between 0 and 1 
df1 = df[df["RPL_THEMES"].between(0, 1)].copy()

print("\nSummary of 'RPL_THEMES' (after filtering):")
print(df1["RPL_THEMES"].describe())


Summary of 'RPL_THEMES' (after filtering):
count    72173.000000
mean         0.499994
std          0.288681
min          0.000000
25%          0.250000
50%          0.500000
75%          0.750000
max          1.000000
Name: RPL_THEMES, dtype: float64


In [8]:
# Divide 'RPL_THEMES' into three classes (low, medium, high) using tertiles 
df1['RPL_THEMES_BIN'] = pd.qcut(df1['RPL_THEMES'], q=3, labels=[0, 1, 2])

# Check the distribution of the new classes
print("\nDistribution of 'RPL_THEMES_BIN':")
print(df1['RPL_THEMES_BIN'].value_counts())



Distribution of 'RPL_THEMES_BIN':
RPL_THEMES_BIN
0    24060
2    24057
1    24056
Name: count, dtype: int64


In [9]:
# Define target and feature variables 
target_variable = 'RPL_THEMES_BIN'
feature_variables = ['EP_POV', 'EP_UNEMP', 'EP_NOHSDP', 'EP_MINRTY', 'EP_AGE65']
X = df1[feature_variables]
y = df1[target_variable].astype(int)  # Convert categorical to integer

# Check the first few rows of feature variables
print("\nFirst few rows of features:")
print(X.head())

# Standardize the data
X = (X - X.mean()) / X.std()

print("\nFirst few rows of target variable:")
print(y.head())

# Check for missing values
print("\nNumber of missing values:")
print(X.isnull().sum())

# Fill missing values with the median  
X = X.fillna(X.median())

# Check missing values again 
print("\nNumber of missing values (after imputation):")
print(X.isnull().sum())


First few rows of features:
     EP_POV  EP_UNEMP  EP_NOHSDP  EP_MINRTY  EP_AGE65
24      8.5       8.9        2.0       20.3      11.2
107     7.8       5.6       11.7       33.9      22.1
198     8.0       2.6        8.7        1.7      21.0
211    11.4       4.7        1.3        3.5      20.6
233    17.9       2.1        8.8        4.2      29.7

First few rows of target variable:
24     1
107    2
198    1
211    0
233    1
Name: RPL_THEMES_BIN, dtype: int64

Number of missing values:
EP_POV       0
EP_UNEMP     0
EP_NOHSDP    0
EP_MINRTY    0
EP_AGE65     0
dtype: int64

Number of missing values (after imputation):
EP_POV       0
EP_UNEMP     0
EP_NOHSDP    0
EP_MINRTY    0
EP_AGE65     0
dtype: int64


In [10]:
# Set float display to 8 decimal places  
pd.options.display.float_format = '{:0.15f}'.format

# Apply KBinsDiscretizer
kbd = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
X_binned = kbd.fit_transform(X)
X_binned = pd.DataFrame(X_binned, columns=feature_variables)

print("\nFirst few rows of data after binning into 3 bins:")
print(X_binned.head())


First few rows of data after binning into 3 bins:
             EP_POV          EP_UNEMP         EP_NOHSDP         EP_MINRTY  \
0 1.000000000000000 2.000000000000000 0.000000000000000 1.000000000000000   
1 0.000000000000000 1.000000000000000 1.000000000000000 1.000000000000000   
2 0.000000000000000 0.000000000000000 1.000000000000000 0.000000000000000   
3 1.000000000000000 1.000000000000000 0.000000000000000 0.000000000000000   
4 2.000000000000000 0.000000000000000 1.000000000000000 0.000000000000000   

           EP_AGE65  
0 0.000000000000000  
1 2.000000000000000  
2 2.000000000000000  
3 2.000000000000000  
4 2.000000000000000  


In [11]:
# Perform Chi-square test 
chi2_stat, p_values = chi2(X_binned, y)

# Create a DataFrame for statistics and p-values
chi2_results = pd.DataFrame({
    'Feature': feature_variables,
    'Chi2 Statistic': chi2_stat,
    'p-value': p_values
})

# Sort by Chi-square statistic and get the top 5 
chi2_top5 = chi2_results.sort_values(by='Chi2 Statistic', ascending=False)

print("\nTop 5 important features based on Chi-square test:") 
print(chi2_top5[['Feature', 'Chi2 Statistic', 'p-value']])


Top 5 important features based on Chi-square test:
     Feature        Chi2 Statistic           p-value
2  EP_NOHSDP 26418.683296464521845 0.000000000000000
0     EP_POV 26048.013400624549831 0.000000000000000
3  EP_MINRTY 13713.022703648970491 0.000000000000000
1   EP_UNEMP 13316.504565824296151 0.000000000000000
4   EP_AGE65  2287.042858585005888 0.000000000000000


In [12]:
# Calculate feature importance using Spearman correlation
spearman_results = []
for feature in feature_variables:
    corr, p = spearmanr(X[feature], y)
    spearman_results.append({
        'Feature': feature,
        'Spearman Correlation': corr,  # Signed correlation 
        'p-value': round(p, 8)
    })

spearman_df = pd.DataFrame(spearman_results)

# Sort by absolute correlation and get the top 5
spearman_top5 = spearman_df.reindex(
    spearman_df['Spearman Correlation'].abs().sort_values(ascending=False).index
)

print("\nTop 5 important features based on Spearman correlation:")  
print(spearman_top5[['Feature', 'Spearman Correlation', 'p-value']])


Top 5 important features based on Spearman correlation:
     Feature  Spearman Correlation           p-value
2  EP_NOHSDP     0.777560089369926 0.000000000000000
0     EP_POV     0.769867580747304 0.000000000000000
1   EP_UNEMP     0.561995184353346 0.000000000000000
3  EP_MINRTY     0.546526127253726 0.000000000000000
4   EP_AGE65    -0.201356601681077 0.000000000000000


In [13]:
# Calculate feature importance using Kendall's Tau 
from scipy.stats import kendalltau

kendall_results = []

for feature in feature_variables:

    corr, p = kendalltau(X[feature], y)
    kendall_results.append({
        'Feature': feature,
        'Kendall Tau': corr,
        'p-value': round(p, 8)
    })

kendall_df = pd.DataFrame(kendall_results)

# Sort by absolute value and get the top 5
kendall_top5 = kendall_df.reindex(
    kendall_df['Kendall Tau'].abs().sort_values(ascending=False).index
)

print("\nTop 5 important features based on Kendall's Tau:") 
print(kendall_top5[['Feature', 'Kendall Tau', 'p-value']])



Top 5 important features based on Kendall's Tau:
     Feature        Kendall Tau           p-value
2  EP_NOHSDP  0.638781154620952 0.000000000000000
0     EP_POV  0.632952621889376 0.000000000000000
1   EP_UNEMP  0.446652595041508 0.000000000000000
3  EP_MINRTY  0.426741083447876 0.000000000000000
4   EP_AGE65 -0.155548319323142 0.000000000000000


In [14]:
# Build Random Forest model 
rf = RandomForestClassifier(random_state=42)
rf.fit(X, y)

# Get feature importances 
rf_importances = pd.Series(rf.feature_importances_, index=feature_variables)
rf_top5 = rf_importances.sort_values(ascending=False)

print("\nTop 5 important features based on Random Forest:") 
print(rf_top5)


Top 5 important features based on Random Forest:
EP_POV      0.298084056150380
EP_NOHSDP   0.295770274360944
EP_MINRTY   0.176786814801612
EP_UNEMP    0.128568353794457
EP_AGE65    0.100790500892607
dtype: float64


In [15]:
from xgboost import XGBClassifier

# Build and train XGBoost model
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xgb.fit(X, y)

# Get feature importances  
xgb_importances = pd.Series(xgb.feature_importances_, index=feature_variables)
xgb_top5 = xgb_importances.sort_values(ascending=False)

print("\nTop 5 important features based on XGBoost:")  
print(xgb_top5)


/Users/hiro/.pyenv/versions/3.9.16/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [21:32:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Top 5 important features based on XGBoost:
EP_POV      0.463402390480042
EP_NOHSDP   0.352279692888260
EP_MINRTY   0.090007804334164
EP_UNEMP    0.059058733284473
EP_AGE65    0.035251379013062
dtype: float32


In [16]:
# Build SVM model
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Create pipeline
svc = make_pipeline(StandardScaler(), SVC(kernel='linear', random_state=42))
svc.fit(X, y)

# Get feature weights
svc_weights = pd.Series(svc.named_steps['svc'].coef_[0], index=feature_variables)
svc_top5 = svc_weights.abs().sort_values(ascending=False)

print("\nTop 5 important features based on SVM:")
print(svc_top5)


Top 5 important features based on SVM:
EP_NOHSDP   2.156098304974876
EP_POV      1.815424765759417
EP_UNEMP    0.635572181303814
EP_MINRTY   0.597628749806063
EP_AGE65    0.180090928876780
dtype: float64


In [17]:
# Build Lasso regression model 
from sklearn.linear_model import LogisticRegression

# Create pipeline 
lasso = make_pipeline(StandardScaler(), LogisticRegression(penalty='l1', solver='liblinear', random_state=42))
lasso.fit(X, y)

# Get feature weights
lasso_weights = pd.Series(lasso.named_steps['logisticregression'].coef_[0], index=feature_variables)
lasso_top5 = lasso_weights.abs().sort_values(ascending=False)
print("\nTop 5 important features based on Lasso regression:")
print(lasso_top5)


Top 5 important features based on Lasso regression:
EP_NOHSDP   3.115316685555746
EP_POV      2.406547358206410
EP_UNEMP    0.919728992228639
EP_MINRTY   0.867641134813764
EP_AGE65    0.284622655895218
dtype: float64


In [18]:
# Build Naive Bayes model
from sklearn.naive_bayes import GaussianNB

# Create pipeline
nb = make_pipeline(StandardScaler(), GaussianNB())
nb.fit(X, y)

# Get feature weights
nb_weights = pd.Series(nb.named_steps['gaussiannb'].theta_[0], index=feature_variables)
nb_top5 = nb_weights.abs().sort_values(ascending=False)

print("\nTop 5 important features based on Naive Bayes:") 
print(nb_top5)


Top 5 important features based on Naive Bayes:
EP_NOHSDP   0.753779592307598
EP_POV      0.745852869003636
EP_MINRTY   0.588756756022015
EP_UNEMP    0.550160000837189
EP_AGE65    0.173679567560513
dtype: float64


In [19]:
# Get top 5 features for each method in ranking order
top_features_rf = rf_importances.sort_values(ascending=False).index.tolist()
top_features_xgb = xgb_importances.sort_values(ascending=False).index.tolist()
top_features_svc = svc_weights.abs().sort_values(ascending=False).index.tolist()
top_features_lasso = lasso_weights.abs().sort_values(ascending=False).index.tolist()
top_features_nb = nb_weights.abs().sort_values(ascending=False).index.tolist()

# Create a rank list from 1 to 5  
rank = list(range(1, 6))

# Create a combined DataFrame  
df_rank = pd.DataFrame({
    'Rank': rank,
    'Random Forest': top_features_rf,
    'XGBoost': top_features_xgb,
    'Lasso': top_features_lasso,
    'SVM': top_features_svc,
    'Naive Bayes': top_features_nb
})

print("\nFeature ranking by each method:")  
print(df_rank)


Feature ranking by each method:
   Rank Random Forest    XGBoost      Lasso        SVM Naive Bayes
0     1        EP_POV     EP_POV  EP_NOHSDP  EP_NOHSDP   EP_NOHSDP
1     2     EP_NOHSDP  EP_NOHSDP     EP_POV     EP_POV      EP_POV
2     3     EP_MINRTY  EP_MINRTY   EP_UNEMP   EP_UNEMP   EP_MINRTY
3     4      EP_UNEMP   EP_UNEMP  EP_MINRTY  EP_MINRTY    EP_UNEMP
4     5      EP_AGE65   EP_AGE65   EP_AGE65   EP_AGE65    EP_AGE65


In [27]:
# Create a rank list from 1 to 5 
rank = [1, 2, 3, 4, 5]

# Get ranked features for each method 
features_kendall_ranked = kendall_top5['Feature'].tolist()
features_chi2_ranked = chi2_top5['Feature'].tolist()
features_spearman_ranked = spearman_top5['Feature'].tolist()

# Create a combined DataFrame 
rank_table = pd.DataFrame({
    'Rank': rank,
    'Chi-Squared Statistic': features_chi2_ranked,
    'Spearman Correlation': features_spearman_ranked,
    'Kendall Tau': features_kendall_ranked
})

# Set index to Rank
rank_table = rank_table.set_index('Rank')

print("\nCombined top 5 features from each method:")  
print(rank_table)


Combined top 5 features from each method:
     Chi-Squared Statistic Spearman Correlation Kendall Tau
Rank                                                       
1                EP_NOHSDP            EP_NOHSDP   EP_NOHSDP
2                   EP_POV               EP_POV      EP_POV
3                EP_MINRTY             EP_UNEMP    EP_UNEMP
4                 EP_UNEMP            EP_MINRTY   EP_MINRTY
5                 EP_AGE65             EP_AGE65    EP_AGE65


### Variable Names
* EP_POV → percentage of persons below the poverty estimate
* EP_NOHSDP → percentage of persons with no high school diploma for those aged 25 and older
* EP_UNEMP → unemployment rate estimate
* EP_MINRTY → percentage minority estimate
* EP_AGE65 → percentage of persons aged 65 and older estimate

* RPL_THEMES → Overall percentile ranking

In [21]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF calculation function 
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data.sort_values(by='VIF', ascending=False)

# Initial VIF calculation 
vif_data = calculate_vif(X)
print("\nInitial VIF:")
print(vif_data)


Initial VIF:
     feature               VIF
0     EP_POV 2.024807641178533
2  EP_NOHSDP 1.915279483833580
3  EP_MINRTY 1.840969557493327
1   EP_UNEMP 1.613642996203697
4   EP_AGE65 1.209273701564790


Since all explanatory variables have a VIF below 5, none require removal.